# Projet maison n° 3

In [3]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas  (par ex. fonction \"concat\" ou méthode \"append\"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation ci-dessus).

In [1]:
# 1) implémentation avec glob + append
import glob

def df_names_us():
    df = pd.DataFrame()
    files = glob.glob('names/*.txt')
    files.sort()
    for filename in files:
        csv = pd.read_csv(filename,
                          names=['name', 'gender', 'births'])
        csv['year'] = int(filename[-8:-4]) # yobAAAA.txt
        df = df.append(csv, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [4]:
%%time
df_us = df_names_us()
df_us.shape

Wall time: 9.47 s


(2020863, 4)

In [4]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


In [5]:
# 2) implémentation avec range + concat
def df_names_us():
    dfs = []
    for year in range(1880, 2021):
        csv = pd.read_csv(f'names/yob{year}.txt',
                          header=None,
                          names=['name', 'gender', 'births'])
        csv['year'] = year
        dfs.append(csv)
    df = pd.concat(dfs, ignore_index=True)
    df = df[['year', 'name', 'gender', 'births']]
    return df

In [6]:
%%time
df_us = df_names_us()
df_us.shape

Wall time: 2.83 s


(2020863, 4)

In [7]:
# head
df_us.head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [8]:
def df_names_fr():
    # genres
    d = {'1': 'M', '2': 'F'}
    # read_csv
    df = pd.read_csv('nat2020_csv.zip',
                      sep=';',
                      header=0,
                      names=['gender', 'name', 'year', 'births'],
                      converters={
                          'gender': d.get,  # lambda x: d[x]
                          'name': str.title
                      })
    # clean
    df = df.loc[(df['name'].str.len() > 1)
                & (df['year'] != 'XXXX')
                & ~df['name'].str.startswith('_')]
    # types
    df['year'] = df['year'].astype(int)
    # ordre colonnes
    df = df[['year', 'name', 'gender', 'births']]
    # tri
    df = df.sort_values(['year', 'gender', 'births', 'name'],
                   ascending=[True, True, False, True])
    df = df.reset_index(drop=True)
    
    return df

In [13]:
dict.get?

In [9]:
%%time
df_fr = df_names_fr()
df_fr.shape

Wall time: 3.11 s


(630407, 4)

In [10]:
# head
df_fr.head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


In [11]:
df_fr.dtypes

year       int32
name      object
gender    object
births     int64
dtype: object

In [12]:
# prénom NA
df_fr.loc[df_fr['name']=='Na']

,year,name,gender,births
404391,2003,Na,F,3


In [14]:
# impact du converters / name
d = {'1': 'M', '2': 'F'}

df = pd.read_csv('nat2020_csv.zip',
                  sep=';',
                  header=0,
                  names=['gender', 'name', 'year', 'births'],)
                  #converters={
                  #    'gender': d.get,
                  #    'name': str.title
                  #})

df['name'].isnull().sum()

2

In [18]:
df.dtypes

gender     int64
name      object
year      object
births     int64
dtype: object

In [15]:
df.isnull().any()

gender    False
name       True
year      False
births    False
dtype: bool

In [16]:
df.loc[df['name'].isnull()]

,gender,name,year,births
563597,2,NaN,2003,3
563598,2,NaN,XXXX,29


In [19]:
# capitalize
'JEAN-MARIE'.capitalize()

'Jean-marie'

In [20]:
# title
'JEAN-MARIE'.title()

'Jean-Marie'

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [21]:
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv",
                     sep=";",
                     na_values='-',
                     decimal=',',
                     skiprows=[0, 1, 3, 4, 5],  # le skiprows permet à l'option "decimal" de fonctionner
                     converters={0: lambda x: pd.to_datetime(x, format='%d/%m/%Y', errors='ignore')})

    # extraction des codes monnaies
    cols = pd.Series(df.columns.tolist()).str.extract('\(([A-Z]{3})\)', expand=True)
    cols.iloc[0] = 'Date'
    df.columns = cols[0]

    # selection des devises
    df = df[['Date'] + devises]

    # drop na
    df = df.dropna()

    # set index
    df = df.set_index('Date')
    
    return df

In [22]:
%%time
df_tx = df_taux_change(['CHF', 'GBP', 'USD'])
df_tx.shape

Wall time: 1.39 s


(5656, 3)

In [24]:
# head
df_tx

,CHF,GBP,USD
Date,,,
2021-02-04,1.0818,0.87693,1.1996
2021-02-03,1.0804,0.88145,1.2017
2021-02-02,1.0808,0.88075,1.2044
2021-02-01,1.0816,0.88200,1.2084
2021-01-29,1.0798,0.88383,1.2136
...,...,...,...
1999-01-08,1.6138,0.70940,1.1659
1999-01-07,1.6165,0.70585,1.1632
1999-01-06,1.6116,0.70760,1.1743


In [25]:
df_tx.index

DatetimeIndex(['2021-02-04', '2021-02-03', '2021-02-02', '2021-02-01',
               '2021-01-29', '2021-01-28', '2021-01-27', '2021-01-26',
               '2021-01-25', '2021-01-22',
               ...
               '1999-01-15', '1999-01-14', '1999-01-13', '1999-01-12',
               '1999-01-11', '1999-01-08', '1999-01-07', '1999-01-06',
               '1999-01-05', '1999-01-04'],
              dtype='datetime64[ns]', name='Date', length=5656, freq=None)

### Tests

In [23]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [24]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [25]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.292s

OK


In [32]:
df_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2020858,2020,Zykell,M,5
2020859,2020,Zylus,M,5
2020860,2020,Zymari,M,5
2020861,2020,Zyn,M,5


In [33]:
2020-1880+1

141

### Session 5 - Un peu de Data Science...

**Question n° 1**

Pourquoi le value_counts() du "gender" donne-t-il un tel écart entre F et M ?

In [26]:
# value_counts du gender

df_us['gender'].value_counts()

F    1192115
M     828748
Name: gender, dtype: int64

In [27]:
# value_counts du gender

df_fr['gender'].value_counts()

F    342873
M    287534
Name: gender, dtype: int64

In [28]:
# pct_change

df_us['gender'].value_counts().pct_change()

F         NaN
M   -0.304809
Name: gender, dtype: float64

In [29]:
# pct_change

df_fr['gender'].value_counts().pct_change()

F         NaN
M   -0.161398
Name: gender, dtype: float64

**Question n° 2**

Pourquoi le value_counts() du "name" donne-t-il ce résultat ?

In [30]:
# value_counts du gender

df_us['name'].value_counts().head(16)

Leslie     282
Lee        282
Jesse      282
Marion     282
Joseph     282
James      282
John       282
Tommie     282
Ollie      282
Francis    282
William    282
Jessie     282
Sidney     282
Johnnie    282
Jean       282
Henry      281
Name: name, dtype: int64

In [31]:
# value_counts du gender

df_fr['name'].value_counts().head(16)

Alix         242
Camille      242
Ange         242
Dominique    238
Claude       235
Hyacinthe    231
Maxime       230
Marie        213
Gaby         210
Cyrille      208
Elie         202
Andréa       201
France       198
Léandre      197
Irène        196
Stéphane     194
Name: name, dtype: int64

**Question n° 3**

Pourquoi le value_counts() du "year" donne-t-il ce résultat ?

In [34]:
# value_counts du gender

df_us['year'].value_counts().head(16)

2008    35084
2007    34964
2009    34715
2006    34096
2010    34085
2011    33919
2012    33761
2013    33304
2014    33269
2015    33145
2016    33031
2017    32634
2005    32554
2018    32178
2004    32051
2019    32030
Name: year, dtype: int64

In [35]:
# value_counts du gender

df_fr['year'].value_counts().head(16)

2014    13989
2012    13899
2013    13862
2011    13749
2015    13686
2016    13610
2017    13569
2018    13502
2010    13364
2019    13355
2020    13078
2009    12422
2008    12268
2007    12132
2006    11929
2005    11387
Name: year, dtype: int64

**Exercice n° 1**

Donnez le prénom qui a été le plus donné lors d'une année.

In [40]:
df_us.loc[df_us['births'].idxmax(), 'name']

'Linda'

In [44]:
df_us.loc[df_us['births'].idxmax()]

year       1947
name      Linda
gender        F
births    99693
Name: 431069, dtype: object

In [41]:
df_us.loc[df_us['births']== df_us['births'].max(), 'name']

431069    Linda
Name: name, dtype: object

In [42]:
df_fr.loc[df_fr['births'].idxmax(), 'name']

'Jean'

In [43]:
df_fr.loc[df_fr['births'].idxmax()]

year       1946
name       Jean
gender        M
births    53584
Name: 95565, dtype: object

**Exercice n° 2**

Donnez la liste des prénoms qui contiennent dans l'ordre a, e, i, o et u (US) ou bien a, e, i et o (FR).

In [45]:
df_us.loc[df_us['name'].str.contains('a.*e.*i.*o.*u'), 'name'].unique()

array(['Laprecious', 'Markevious', 'Laderious', 'Jakevious',
       'Quanterious', 'Quanterrious', 'Latrevious', 'Jaterrious',
       'Jamerious', 'Jaderious', 'Marquevious', 'Laterrious', 'Jaterious',
       'Dametrious'], dtype=object)

In [47]:
df_fr.loc[df_fr['name'].str.contains('a.*e.*i.*o'), 'name'].unique()

array(['Marie-Philomene', 'Marie-Victorine', 'Marie-Victoire',
       'Marie-Simone', 'Marcelino', 'Valerio', 'Valentino', 'Valeriano',
       'Saverio', 'Marie-Nicole', 'Marie-Violaine', 'Marie-Mimose',
       'Marie-Victoria', 'Marcellino', 'Maria-Conception',
       'Marie-Christophe', 'Laurentino', 'Charles-Victor',
       'Charles-Nicolas', 'Marie-Violette', 'Maelio'], dtype=object)

### Méthodes de reshaping (1)

#### pivot_table

La méthode pivot_table() prend en argument :
- values : valeurs qui doivent être agrégées selon les modalités de la colonne passée en "index" et de la colonne passée en "columns"
- index : colonne(s) dont les valeurs vont servir d'index à la table pivot
- columns : colonne(s) dont les valeurs vont servir de colonnes à la table pivot
- aggfunc : fonction d'aggrégation des values, par défaut 'mean', 'median', 'min', 'max', 'count', 'sum', 'nunique', et n'importe quelle lambda ou liste de fonctions.

On obtient NaN s'il n'y a pas d'occurence croisée.

Excel = tableau croisé dynamique.

In [48]:
# exemple
df = pd.DataFrame([{'A': 1,'B': 1, 'C': 1},
                   {'A': 1,'B': 1, 'C': 2},
                   {'A': 1,'B': 2, 'C': -1},
                   {'A': 2,'B': 1, 'C': 4},
                   {'A': 2,'B': 1, 'C': 5},
                  ])

df

,A,B,C
0,1,1,1
1,1,1,2
2,1,2,-1
3,2,1,4
4,2,1,5


In [53]:
# exemple
tab = df.pivot_table(values='C',
              index='A',
              columns='B')
tab

B,1,2
A,,
1,1.5,-1.0
2,4.5,NaN


In [55]:
type(tab)

pandas.core.frame.DataFrame

In [56]:
tab.index

Int64Index([1, 2], dtype='int64', name='A')

In [57]:
tab.columns

Int64Index([1, 2], dtype='int64', name='B')

In [52]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc=tuple)

B,1,2
A,,
1,"(1, 2)","(-1,)"
2,"(4, 5)",NaN


In [50]:
# exemple
df.pivot_table(values='C',
              index='A',
              columns='B',
              aggfunc='sum')

B,1,2
A,,
1,3.0,-1.0
2,9.0,NaN


**Exercice n° 3**

Calculez une table pivot avec le nombre total de naissances par année et par genre.

In [58]:
df_us.pivot_table(values='births',
                  index='year',
                  columns='gender',
                  aggfunc='sum')

gender,F,M
year,,
1880,90994,110490
1881,91953,100738
1882,107847,113686
1883,112319,104625
1884,129019,114442
...,...,...
2016,1768360,1893917
2017,1722140,1846154
2018,1695654,1810309


In [59]:
df_us.loc[(df_us['gender']=='F')&(df_us['year']==1880), 'births'].sum()

90994

In [60]:
df_fr.pivot_table(values='births',
                  index='year',
                  columns='gender',
                  aggfunc='sum')

gender,F,M
year,,
1900,235926,176138
1901,255718,194622
1902,259689,203023
1903,259687,206072
1904,263010,212710
...,...,...
2016,344729,365837
2017,337727,358104
2018,331947,352992


In [73]:
df_fr.pivot_table(values='births',
                  index='year',
                  #columns='gender',
                  aggfunc='sum')

,births
year,
1900,412064
1901,450340
1902,462712
1903,465759
1904,475720
...,...
2016,710566
2017,695831
2018,684939


**Exercice n° 4**

Calculez une table pivot avec la diversité des prénoms (nombre de prénoms distincts) par année et par genre.

In [63]:
tab1 = df_us.pivot_table(values='name',
                  index='year',
                  columns='gender',
                  aggfunc='count')

In [64]:
tab2 = df_us.pivot_table(values='name',
                  index='year',
                  columns='gender',
                  aggfunc='nunique')

In [67]:
(tab1 == tab2).all().all()

True

In [69]:
df_us.pivot_table(values='name',
                  index='year',
                  columns='gender',
                  aggfunc='count')

gender,F,M
year,,
1880,942,1058
1881,938,996
1882,1028,1099
1883,1054,1030
1884,1172,1125
...,...,...
2016,18847,14184
2017,18404,14230
2018,18105,14073


In [70]:
df_fr.pivot_table(values='name',
                  index='year',
                  columns='gender',
                  aggfunc='count')

gender,F,M
year,,
1900,983,723
1901,1010,719
1902,1019,721
1903,1020,737
1904,1013,765
...,...,...
2016,7235,6375
2017,7217,6352
2018,7119,6383


In [71]:
df_fr.pivot_table(values='name',
                  index='year',
                  #columns='gender',
                  aggfunc='count')

,name
year,
1900,1706
1901,1729
1902,1740
1903,1757
1904,1778
...,...
2016,13610
2017,13569
2018,13502


In [72]:
df_fr.pivot_table(values='name',
                  index='year',
                  #columns='gender',
                  aggfunc='nunique')

,name
year,
1900,1594
1901,1611
1902,1621
1903,1646
1904,1673
...,...
2016,13201
2017,13158
2018,13082


#### crosstab

crosstab() est une fonction de reshaping qui prend 2 colonnes d'un DataFrame en argument et produit le décompte croisé des occurrences.

On obtient 0 s'il n'y a pas d'occurence croisée.

In [74]:
# exemple
pd.crosstab(df['A'], df['B'])

B,1,2
A,,
1,2,1
2,2,0


In [75]:
# initial
df_us['initial'] = df_us['name'].str[0].str.upper()
# terminal
df_us['terminal'] = df_us['name'].str[-1].str.upper()

df_us.head()

,year,name,gender,births,initial,terminal
0,1880,Mary,F,7065,M,Y
1,1880,Anna,F,2604,A,A
2,1880,Emma,F,2003,E,A
3,1880,Elizabeth,F,1939,E,H
4,1880,Minnie,F,1746,M,E


In [82]:
# crosstab
pd.crosstab(df_us['initial'], df_us['terminal'],
           #normalize='columns',
           margins=True)#*100

terminal,A,B,C,D,E,F,G,H,I,J,...,R,S,T,U,V,W,X,Y,Z,All
initial,,,,,,,,,,,,,,,,,,,,,
A,65093,385,730,3197,30637,516,214,11554,8487,308,...,6573,9950,2435,670,324,427,605,8900,784,201227
B,12925,238,154,3002,15927,44,183,2481,2345,78,...,2590,2017,3839,345,53,239,293,9130,323,82161
C,29262,424,375,2117,31441,203,796,1916,3777,64,...,5700,7022,2231,169,7,35,73,11891,752,138435
D,32017,43,1082,3088,26379,43,367,3798,4607,102,...,2187,8230,1404,190,154,418,182,8659,385,135629
E,23177,147,505,2483,17818,50,266,4343,2079,53,...,4676,2879,3437,458,14,54,171,4548,133,92862
F,8404,0,287,1509,5637,0,191,959,478,6,...,1237,2116,796,60,0,1,328,2156,685,34559
G,10648,63,19,2547,12074,198,506,1017,1862,24,...,2254,2426,1940,19,199,0,2,4581,25,58168
H,6029,220,20,2662,6535,98,491,2514,2222,8,...,2460,1787,2308,194,7,114,85,5910,124,49934
I,10629,4,754,415,3608,15,227,1730,1068,9,...,883,1655,204,205,2,0,0,1725,302,31443


In [85]:
# pivot_table
df_us.pivot_table(values='name',  # ou "births"
                  index='initial',
                  columns='terminal',
                  aggfunc='count').fillna(0).astype(int)

terminal,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
initial,,,,,,,,,,,,,,,,,,,,,
A,65093,385,730,3197,30637,516,214,11554,8487,308,...,69,6573,9950,2435,670,324,427,605,8900,784
B,12925,238,154,3002,15927,44,183,2481,2345,78,...,0,2590,2017,3839,345,53,239,293,9130,323
C,29262,424,375,2117,31441,203,796,1916,3777,64,...,0,5700,7022,2231,169,7,35,73,11891,752
D,32017,43,1082,3088,26379,43,367,3798,4607,102,...,26,2187,8230,1404,190,154,418,182,8659,385
E,23177,147,505,2483,17818,50,266,4343,2079,53,...,30,4676,2879,3437,458,14,54,171,4548,133
F,8404,0,287,1509,5637,0,191,959,478,6,...,35,1237,2116,796,60,0,1,328,2156,685
G,10648,63,19,2547,12074,198,506,1017,1862,24,...,0,2254,2426,1940,19,199,0,2,4581,25
H,6029,220,20,2662,6535,98,491,2514,2222,8,...,13,2460,1787,2308,194,7,114,85,5910,124
I,10629,4,754,415,3608,15,227,1730,1068,9,...,54,883,1655,204,205,2,0,0,1725,302


In [86]:
# Z x Z
df_us.loc[(df_us['initial'] == 'Z') & (df_us['terminal'] == 'Z')]

,year,name,gender,births,initial,terminal
1332956,1999,Zyquez,M,5,Z,Z
1551477,2006,Zyquez,M,6,Z,Z
1583824,2007,Zyquez,M,8,Z,Z
1616597,2008,Zyquez,M,11,Z,Z
1721482,2011,Zyquez,M,8,Z,Z
1826566,2014,Zoraiz,M,5,Z,Z
1854399,2015,Zoraiz,M,9,Z,Z
1889284,2016,Zoraiz,M,7,Z,Z
1921906,2017,Zoraiz,M,7,Z,Z
1952287,2018,Zoraiz,M,9,Z,Z


In [87]:
# Z x Z
df_us.loc[df_us['name'].str.startswith('Z') & df_us['name'].str.endswith('z')]

,year,name,gender,births,initial,terminal
1332956,1999,Zyquez,M,5,Z,Z
1551477,2006,Zyquez,M,6,Z,Z
1583824,2007,Zyquez,M,8,Z,Z
1616597,2008,Zyquez,M,11,Z,Z
1721482,2011,Zyquez,M,8,Z,Z
1826566,2014,Zoraiz,M,5,Z,Z
1854399,2015,Zoraiz,M,9,Z,Z
1889284,2016,Zoraiz,M,7,Z,Z
1921906,2017,Zoraiz,M,7,Z,Z
1952287,2018,Zoraiz,M,9,Z,Z
